In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

In [ ]:
# Load your dataset
data_path = '/content/drive/MyDrive/bio_summary_final.csv'  
df = pd.read_csv(data_path)

In [5]:
# Convert the DataFrame to a Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [6]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [7]:
# Tokenization function with padding
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["longtext"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)  # Adding padding

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, padding="max_length", truncation=True)  # Adding padding

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
print(df.columns)

Index(['longtext', 'summary', 'Unnamed: 2'], dtype='object')


In [ ]:
# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  
)

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,2.620431
2,No log,2.554500
3,No log,2.533156


TrainOutput(global_step=375, training_loss=2.9181168619791666, metrics={'train_runtime': 541.4695, 'train_samples_per_second': 2.759, 'train_steps_per_second': 0.693, 'total_flos': 1023027484557312.0, 'train_loss': 2.9181168619791666, 'epoch': 3.0})

In [14]:
model.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')

('/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer.json')

In [ ]:
# Define the path 
model_save_path = '/content/drive/MyDrive/flan_t5_finetuned_model_2nd' 

In [16]:
# Save the model
trainer.model.save_pretrained(model_save_path)

In [17]:
# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer.json')

In [18]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00


In [19]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3afba9889df77b1e46f43ef88e36b274a915b6d22316566ac903f3a5de65fc6b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [20]:
import evaluate

# Load the ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip white spaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}  # Convert to percentages

    return result


In [21]:
import rouge_score
print("ROUGE Score Library Imported Successfully")

ROUGE Score Library Imported Successfully


In [22]:
!pip show rouge_score

Name: rouge_score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


In [23]:
from rouge_score import rouge_scorer

# Initialize a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Test the scorer with example text
text1 = (
"The human ear is divided into three parts: the outer ear, middle ear and inner ear. Outer wear consists of the piano and auditory canal. Middle ear (tympanic cavity) is an air-filled cavity within the temporal bone. Three very small bones (ear ossicles) called malleus, incus and stapes extend across the middle ear from tympanic membrane to the oval window. Three very small bones (ear ossicles) called malleus, incus and stapes extend across the middle ear from tympanic membrane to the oval window. Inner ear is formed from a network of channels and cavities in temporal bone which are called bony labyrinth. Cochlear duct is a part of the membranous labyrinth and filled with endolymph. It contains supporting cells and specialized cochlear hair cells containing mechanoreceptors (auditory receptors) of the ear. Hair cells synapse with the dendrites of sensory neurons that combine to form the auditory nerve which transmits auditory information to the brain. The cochlear duct is a part of the membranous labyrinth and filled with endolymph. It contains supporting cells and specialized cochlear hair cells containing mechanoreceptors (auditory receptors).")

text2 = """
The human ear is divided into three parts: outer, middle, and inner. The outer ear consists of the piano and auditory canal, which is a tube lined with hairy skin and sweat glands secreting ear wax. The middle ear is an air-filled cavity within the temporal bone, with two oval and round windows covered by small bones called stapes and fibrous tissue. The inner ear is formed from a network of channels and cavities called the bony labyrinth, which is lined with fluid-filled membranes called membranous labyrinth. The inner ear is composed of three main regions: vestibule, three semicircular canals, and cochlea. The vestibular canal and tympanic canal are continuous and filled with perilymph. The cochlear duct, a part of the membranous labyrinth, contains the basilar membrane, which bears the organ of Corti and contains supporting cells and specialized cochlear hair cells containing auditory receptors."""

# Calculate ROUGE scores
scores = scorer.score(text1, text2)

print(scores)


{'rouge1': Score(precision=0.7414965986394558, recall=0.5989010989010989, fmeasure=0.6626139817629179), 'rouge2': Score(precision=0.4041095890410959, recall=0.3259668508287293, fmeasure=0.3608562691131499), 'rougeL': Score(precision=0.54421768707483, recall=0.43956043956043955, fmeasure=0.48632218844984804)}
